In [1]:
from typing import Literal
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
import pandas as pd
import os
from dotenv import load_dotenv

/Users/dogiheon/Desktop/invigoworks/.venv/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3699: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
load_dotenv(verbose=True)

# 1) 출력 스키마 정의
class Sentiment(BaseModel):
    label: Literal["negative", "neutral", "positive"] = Field(..., description="감성 라벨")

parser = JsonOutputParser(pydantic_object=Sentiment)

# 2) 프롬프트 템플릿
system_text = """뉴스 감성 분석기다.
- 사실 전달형/중립적 보도 톤은 기본값으로 neutral 처리한다.
- 명백한 악재(적자 확대, 리콜, 규제 불이익, 급락 등)는 negative.
- 명백한 호재(사상 최대 실적, 대규모 투자 유치/수주, 급등 등)는 positive.
- 혼재할 경우 기사 전체 톤 기준으로 단 하나의 라벨만 선택한다.
- 반드시 JSON 한 줄만 출력한다.
출력 스키마: {{"label":"negative|neutral|positive"}}
"""

human_text = """
본문: {body}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_text),
        ("human", human_text),
    ]
).partial(format_instructions=parser.get_format_instructions())

# 3) 모델 정의 (환경변수 OPENAI_API_KEY 필요)
OPENAI_API_KEY=os.getenv('OPENAI_API_KEY')

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, api_key=OPENAI_API_KEY)  # 임의 모델. 교체 가능

# 4) 체인 구성: Prompt -> LLM -> JSON 파서
chain = prompt | llm | parser

In [3]:
def parseResponse(label):
    if label=="negative":
        return -1
    elif label=="neutral":
        return 0
    else:
        return 1

In [4]:
start=856
end=856

page=start

for page in range(start,end+1):
    df=pd.read_csv(f"news_list/page{page}.csv")

    news_list=df["news"].to_list()

    inputs=[]

    labels=[0]*len(news_list)

    for news in news_list:
        inputs.append({"body":news})

    async for idx, out in chain.abatch_as_completed(inputs, config={"max_concurrency": 16}):
        labels[idx]=parseResponse(out["label"])

    df["label"]=labels
    df.to_csv(f"news_list_labeled/page{page}.csv", index=False)
    print(f"page {page} finished!")

page 856 finished!
